In [1]:
# !gdown --id 1YmVjZ2gRtdDXrdrWui4TR-IVoe6G_Guk
!gdown --id 1_efNqv5Idm0qgRWZbwEHE2V6Dd1Kjim-



/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1_efNqv5Idm0qgRWZbwEHE2V6Dd1Kjim-
From (redirected): https://drive.google.com/uc?id=1_efNqv5Idm0qgRWZbwEHE2V6Dd1Kjim-&confirm=t&uuid=62e0e870-8509-408d-be65-0c5f569884f9
To: /content/model_BERT.pkl
100% 438M/438M [00:05<00:00, 84.2MB/s]


In [2]:

import pickle
#load the model
with open('/content/model_BERT.pkl','rb') as f:
  model = pickle.load(f)


In [3]:
import json

def load_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

###upload the test file in colab itself---------------------------

test_data = load_json_file('/content/test_file.json')


In [4]:
test_X=[]
test_Y=[]
for i in test_data:
    test_X.extend([j for j in i['utterances']])
    test_Y.extend([j for j in i['emotions']])

In [5]:
emo = { 'anger': [1, 0, 0, 0, 0, 0, 0], 'disgust': [0, 1, 0, 0, 0, 0, 0],'fear': [0, 0, 1, 0, 0, 0, 0],'joy': [0, 0, 0, 1, 0, 0, 0],'neutral': [0, 0, 0, 0, 1, 0, 0], 'sadness': [0, 0, 0, 0, 0, 1, 0],'surprise': [0, 0, 0, 0, 0, 0, 1]}
for i in range(len(test_Y)):
    k=test_Y[i]
    test_Y[i]=emo[k]

In [6]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

test_X = [preprocess_text(sentence) for sentence in test_X]


In [8]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
import torch
import numpy as np
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(sentences, labels):
  tokenized_data = tokenizer(sentences, padding="max_length", truncation=True)
  input_ids = torch.tensor(tokenized_data['input_ids'], dtype=torch.long)
  attention_mask = torch.tensor(tokenized_data['attention_mask'], dtype=torch.long)
  labels = torch.tensor(labels, dtype=torch.long)
  return TensorDataset(input_ids, attention_mask, labels)


test_data = preprocess_data(test_X, test_Y)
test_loader = DataLoader(test_data,batch_size=32)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
import torch
import numpy as np
# from torch.nn.functional import to_categorical
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
model.eval()
predictions = []
true_labels = []
ohe = { 'anger': [1, 0, 0, 0, 0, 0, 0], 'disgust': [0, 1, 0, 0, 0, 0, 0],'fear': [0, 0, 1, 0, 0, 0, 0],'joy': [0, 0, 0, 1, 0, 0, 0],'neutral': [0, 0, 0, 0, 1, 0, 0], 'sadness': [0, 0, 0, 0, 0, 1, 0],'surprise': [0, 0, 0, 0, 0, 0, 1]}
eho = {0: [1, 0, 0, 0, 0, 0, 0], 1: [0, 1, 0, 0, 0, 0, 0], 2: [0, 0, 1, 0, 0, 0, 0], 3: [0, 0, 0, 1, 0, 0, 0], 4: [0, 0, 0, 0, 1, 0, 0], 5: [0, 0, 0, 0, 0, 1, 0], 6: [0, 0, 0, 0, 0, 0, 1]}

with torch.no_grad():
    i=0
    for batch in test_loader:
        print(f"Batch {i} of {len(test_loader)}")
        i+=1
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions.extend(torch.argmax(outputs.logits, axis=1).tolist())
        true_labels.extend(labels.tolist())



Batch 0 of 228
Batch 1 of 228
Batch 2 of 228
Batch 3 of 228
Batch 4 of 228
Batch 5 of 228
Batch 6 of 228
Batch 7 of 228
Batch 8 of 228
Batch 9 of 228
Batch 10 of 228
Batch 11 of 228
Batch 12 of 228
Batch 13 of 228
Batch 14 of 228
Batch 15 of 228
Batch 16 of 228
Batch 17 of 228
Batch 18 of 228
Batch 19 of 228
Batch 20 of 228
Batch 21 of 228
Batch 22 of 228
Batch 23 of 228
Batch 24 of 228
Batch 25 of 228
Batch 26 of 228
Batch 27 of 228
Batch 28 of 228
Batch 29 of 228
Batch 30 of 228
Batch 31 of 228
Batch 32 of 228
Batch 33 of 228
Batch 34 of 228
Batch 35 of 228
Batch 36 of 228
Batch 37 of 228
Batch 38 of 228
Batch 39 of 228
Batch 40 of 228
Batch 41 of 228
Batch 42 of 228
Batch 43 of 228
Batch 44 of 228
Batch 45 of 228
Batch 46 of 228
Batch 47 of 228
Batch 48 of 228
Batch 49 of 228
Batch 50 of 228
Batch 51 of 228
Batch 52 of 228
Batch 53 of 228
Batch 54 of 228
Batch 55 of 228
Batch 56 of 228
Batch 57 of 228
Batch 58 of 228
Batch 59 of 228
Batch 60 of 228
Batch 61 of 228
Batch 62 of 228
Ba

In [10]:
t=[]
for i in predictions:
    t.append(eho[i])
predictions=t

In [11]:
from sklearn.metrics import f1_score

correct = sum([1 if p == t else 0 for p, t in zip(predictions, true_labels)])
accuracy = correct / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")


f1 = f1_score(true_labels, predictions, average='weighted')
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.5568
F1 Score: 0.5148
